## Exercise

The company `SparkyCO` have noticed more than usual servers connectivity disruptions. You were given a task to investigate that and monitor all requests processed by the severs.

You need to process sever access logs, monitor incoming traffic and detect any anomaly

Logs schema:
`host timestamp method endpoint protocol status content_size`

1. Read incoming stream data using
2. Parse incoming log (use provided regex expressions or any other method)
3. Apply tumbling window aggregation of `15 minutes` and add 5 minut offset
4. Filter "localhost" endpoints
5. Order by "count"
5. add watermark
6. add boolean column "possible_threat", if `count` is over 1_000
6. add checkpoint
7. Write output to a file output

## Setup


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import regexp_extract

In [ ]:
spark = SparkSession.builder.appName("Log Analysis").getOrCreate()

### Generate data
For out practice purposes, we will generate mockup Apache Access logs.
>1. Open terminal and change directory to `data` folder
>2. In terminal, run command `python log_generator apache_access_schema.yaml`, add flag `-t` to truncate previously created logs
>3. Based on provide .yaml file, new `log.txt.{id}` file should appear every 5s with 50 lines of random logs in `log/apache_access` directory

In [ ]:
# regex patterns to parse logs

host_pattern = r'(^\S+\.[\S+\.]+\S+)\s'
time_pattern = r'\[(\d{2}/\w{3}/\d{4}:\d{2}:\d{2}:\d{2}:\d{4})]'
general_pattern = r'\"(\S+)\s(\S+)\s*(\S*)\"'
status_pattern = r'\s(\d{3})\s'
content_size_pattern = r'\s(\d+)$'

## Practice

1. Create DataFrame representing the stream of raw log data lines ariving to `/log/apache_access` directory

In [ ]:
access_lines = spark.readStream.text("../data/log/apache_access")

2. Parse log data to DataFrame

In [ ]:


# logsDF = access_lines.select(regexp_extract('value', hostExp, 1).alias('host'),
#                         regexp_extract('value', timeExp, 1).alias('timestamp'),
#                         regexp_extract('value', generalExp, 1).alias('method'),
#                         regexp_extract('value', generalExp, 2).alias('endpoint'),
#                         regexp_extract('value', generalExp, 3).alias('protocol'),
#                         regexp_extract('value', statusExp, 1).cast('integer').alias('status'),
#                         regexp_extract('value', contentSizeExp, 1).cast('integer').alias('content_size'))

3. Count every access status code

In [ ]:
statusCountsDF=logsDF.groupBy("status").count()

4. Kick off our streaming query, dumping results to the console

In [ ]:
query = statusCountsDF.writeStream.outputMode("complete").format("console").queryName("status_counts").start()

5. Classroom cleanup
IMPORTANT: Kill running log_generator script in terminal

In [ ]:
query.stop()
spark.stop()